In [2]:
import numpy as np

In [5]:
obstacle = np.load("preprocessing/pysocialforce_obstacles_meter/00_Zurich_HB_anchored.npy")
print(obstacle)

[[  6.46999194   6.87436644 242.62469781 242.62469781]
 [  6.46999194   6.46999194 242.62469781 242.22032332]
 [  6.87436644   7.27874093 242.62469781 242.62469781]
 ...
 [243.4334468  243.8378213    0.4043745    0.4043745 ]
 [243.4334468  243.8378213    0.           0.        ]
 [243.8378213  243.8378213    0.4043745    0.        ]]
